In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread

In [2]:
def mse(y, t):
    return 0.5 * np.sum((y-t)**2)

In [5]:
y = [.1, .05, .6, .0, .05, .1, .0, .1, .0, .0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [7]:
mse(np.array(y), np.array(t))

0.09750000000000003

In [8]:
y = [.1 , .05, .1, .0, .05, .1, .0, .6, .0, .0]

In [9]:
mse(np.array(y), np.array(t))

0.5975

In [10]:
def cross_entropy_error(y,t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

In [12]:
cross_entropy_error(np.array(y), np.array(t))

2.302584092994546

In [14]:
y = [.1, .05, .6, .0, .05, .1, .0, .1, .0, .0]
cross_entropy_error(np.array(y), np.array(t))

0.510825457099338

In [15]:
from dataset.mnist import load_mnist

In [16]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize= True, one_hot_label=True)

In [17]:
print(x_train.shape)
print(t_train.shape)

(60000, 784)
(60000, 10)


In [19]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size) ### randomly selects n(batch_size) number of values withint range(train_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [23]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)    ### ground truth
        y = y.reshape(1, y.size)    ### output of the nn
    batch_size = y.shape[0]
    return -np.sum(t * np.log( y + 1e-7)) / batch_size

In [24]:
### crosss entropy that ignores when t == 0
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y[np.arange(y), t] + 1e-7)) / batch_size 

In [25]:
def numerical_diff(f, x):
    h = 1e-4 # 0.0001
    return (f(x+h) - f(x-h)) / (2 * h)

In [34]:
def numerical_gradient(f,x):
    h = 1e-4
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        temp_val = x[idx]
        x[idx] = temp_val + h
        fxh1 = f(x)
        
        x[idx] = temp_val - h 
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = temp_val
    return grad

def gradient_descent(f , init_x, lr = 0.01, step_num = 100):
    x = init_x
    for i in range(step_num):
        grad = numerical_gradient(f,x)
        x -= lr * grad
    return x

In [35]:
def function_2(x1):
    return 3.0 ** 2.0 + x1 * x1

numerical_diff(function_2, np.array([3.0, 4.0]))

array([6., 8.])

In [36]:
numerical_diff(function_2, np.array([0.0, 2.0]))

array([0., 4.])

In [37]:
def function_2(x):
    return x[0]**2 + x[1]**2

init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x = init_x, lr = 0.1, step_num = 100)

array([-6.11110793e-10,  8.14814391e-10])

In [38]:

init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x = init_x, lr = 10.0, step_num = 100)

array([-2.58983747e+13, -1.29524862e+12])

In [39]:

init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x = init_x, lr = 1e-10, step_num = 100)

array([-2.99999994,  3.99999992])

### SimpleNet

In [42]:
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

In [47]:
class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3)
    
    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y ,t)
        
        return loss

net = simpleNet()
x = np.array([.6, .9])
p = net.predict(x)
print(p)
t = np.array([0, 0, 1])
net.loss(x, t)

[-0.38947585 -1.09236204  0.97305415]


0.3240942124139277

In [48]:
def f(W):
    return net.loss(x, t)

dW = numerical_gradient(f, net.W)
print(dW)

[[ 0.11108605  0.05500472 -0.16609077]
 [ 0.16662908  0.08250708 -0.24913616]]


### Two Layer Net

In [49]:
from common.functions import *

In [58]:
class TwoLayerNet :
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        self.params={}
        self.params["W1"] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params["b1"] = np.zeros(hidden_size)
        self.params["W2"] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params["b2"] = np.zeros(output_size)
        
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
    
    def loss(self, x , t):
        y = self.predict(x)
        
        return cross_entropy_error(y ,t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis = 1)
        t = np.argmax(t, axis = 1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x ,t ):
        loss_W  = lambda w : self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads    

In [63]:
net = TwoLayerNet(input_size = 784, hidden_size = 100, output_size = 10)
net.params['W1'].shape
net.params['b1'].shape
net.params['W2'].shape
net.params['b2'].shape

x = np.random.rand(100, 784)
t = np.random.rand(100, 10)
y = net.predict(x)

grads = net.numerical_gradient(x, t)
grads['W1'].shape

(784, 100)

In [64]:
from two_layer_net import TwoLayerNet

In [65]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label=True)

In [69]:
train_loss_list = []

iters_num = 100
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

In [70]:
network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)
for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grad = network.numerical_gradient(x_batch, t_batch)
    
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)


In [71]:
print(train_loss_list)

[2.2942085045410003, 2.299196964863279, 2.29881833971639, 2.290792079677234, 2.291239198747211, 2.2920856819400304, 2.294699735874695, 2.300572778116304, 2.296810310616586, 2.2893518443909837, 2.2866846534687086, 2.298575572939326, 2.2899827204639056, 2.3023190389825796, 2.292305551597616, 2.296278490589219, 2.296353509540519, 2.2931554121685913, 2.2892911261462263, 2.2912094720676586, 2.2829577756382857, 2.283026794536466, 2.295948079157912, 2.284042024673277, 2.289530211412664, 2.278123164167957, 2.296799152382205, 2.2844462999229638, 2.2980498581131337, 2.2807140366164433, 2.288587089843959, 2.2894837118444116, 2.2909801886853716, 2.2778824440370733, 2.285606381057649, 2.282718461375895, 2.260767413276341, 2.2985570579206196, 2.2937962041457847, 2.2911269130414005, 2.28047882439221, 2.290788761584613, 2.275772352972058, 2.280587851483584, 2.2816230619221747, 2.305103786449184, 2.2781610869481512, 2.2787317253929062, 2.285572189187432, 2.2966643602111767, 2.2665108625680825, 2.299582

## validation

In [78]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 100
train_size = x_train.shape[0]
batch_size = 10
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size , 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    
    grad = network.numerical_gradient(x_batch, t_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        
        print("train acc , test acc : " + str(train_acc) + ", " +  str(test_acc))


train acc , test acc : 0.09863333333333334, 0.0958


train acc , test acc : 0.09863333333333334, 0.0958
